# Solutions 2

In [1]:
import numpy as np

In [2]:
class Call:
    
    def __init__(self, strike):
        self.strike = strike

    def __call__(self, spot):
        return np.maximum(spot - self.strike, 0.)

In [3]:
class VanillaOption:
    
    def __init__(self, maturity, payoff):
        self.maturity = maturity
        self.payoff = payoff


In [4]:
class BlackScholesProcess:
    def __init__(self, spot, rate, vol):
        self.spot = spot
        self.rate = rate
        self.vol = vol


    def evolve(self, time, norms):
        ST = self.spot * np.exp( (self.rate - 0.5 *self.vol**2)*time + self.vol*np.sqrt(time)*norms )
        return ST
        

In [5]:
class FixedRateYieldCurve:
    
    def __init__(self, rate):
        self.rate = rate
      

    def discount(self, maturity):
        return np.exp(-self.rate * maturity)

In [6]:
def mc_price_european(process, option, yield_curve, n_paths):
    std_norms = np.random.normal(size=n_paths)
    S_T = process.evolve(option.maturity, std_norms)
    V_T = yield_curve.discount(option.maturity) * option.payoff(S_T)
    V_0 = np.average(V_T)   

    return V_0

In [7]:
T = 5.
S_0 = 100.
vol = 0.2
r = 0.02
K = 100
K2 = 120
N = 10000

In [8]:
yield_curve = FixedRateYieldCurve(r)
rand_process = BlackScholesProcess(S_0, r, vol)
call_payoff = Call(K)
call_option = VanillaOption(T, call_payoff)

In [9]:
c = mc_price_european(rand_process, call_option, yield_curve, N)
print(c)

21.91524141421885


In [10]:
class Put:
    def __init__(self, strike):
        self.strike = strike

    def __call__(self, spot):
        return np.maximum(self.strike - spot, 0.)    

In [11]:
class DoubleDigital:
    def __init__(self, low, high):
        self.low = low
        self.high = high

    def __call__(self, spot):
        payoff = np.zeros_like(spot)    
        indices_low = np.where(self.low <= spot)
        indices_high = np.where(self.high >= spot)
        payoff[indices_low and indices_high] = 1
        return payoff

In [12]:
put_payoff = Put(K)
put_option = VanillaOption(T, put_payoff)
put_price = mc_price_european(rand_process, put_option, yield_curve, N)
print(put_price)

12.782182647182157


In [13]:
dbl_digit_payoff = DoubleDigital(K, K2)
double_digital_option = VanillaOption(T, dbl_digit_payoff)
dbl_digital_price = mc_price_european(rand_process, double_digital_option, yield_curve, N)
print(dbl_digital_price)

0.5976451146127513
